In [1]:
pip install --upgrade "numpy<2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [2]:
!pip install torch==2.0.1 torchtext==0.15.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [3]:
!pip install portalocker

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset

In [5]:
import os
import glob

# Download and extract the IMDB dataset
if not os.path.exists("/content/aclImdb"):
    !wget -nc https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    !tar -xzf aclImdb_v1.tar.gz

def load_imdb_data(path, label):
    """Loads text data from a given path and assigns a label."""
    texts = []
    for file in glob.glob(os.path.join(path, '*.txt')):
        with open(file, 'r', encoding='utf-8') as f:
            texts.append((f.read(), label))
    return texts

# Define paths inside Colab's workspace
train_pos = load_imdb_data('/content/aclImdb/train/pos', 'pos')
train_neg = load_imdb_data('/content/aclImdb/train/neg', 'neg')
test_pos = load_imdb_data('/content/aclImdb/test/pos', 'pos')
test_neg = load_imdb_data('/content/aclImdb/test/neg', 'neg')

# Combine positive and negative samples
train_data = train_pos + train_neg
test_data = test_pos + test_neg

# Shuffle the dataset (important for training)
import random
random.shuffle(train_data)
random.shuffle(test_data)

# Print dataset size
print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

--2025-03-21 15:03:07--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  17.0MB/s    in 11s     

2025-03-21 15:03:18 (7.16 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]

Training samples: 25000
Test samples: 25000


In [37]:
from torch.utils.data import DataLoader, Dataset

class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, data, vocab, tokenizer):
        self.data = data
        self.vocab = vocab
        self.tokenizer = tokenizer
        self.label_map = {"negative": 0, "positive": 1, "neg": 0, "pos": 1}  # Add 'neg' and 'pos'

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text, label = self.data[idx]

        # Convert text to token IDs
        tokenized_text = self.tokenizer(text)
        text_tensor = torch.tensor([self.vocab[token] for token in tokenized_text], dtype=torch.int64)

        # Convert label from string to int
        label_tensor = torch.tensor(self.label_map[label], dtype=torch.float32)

        return text_tensor, label_tensor



def collate_batch(batch):
    texts, labels = zip(*batch)
    texts = pad_sequence(texts, batch_first=True, padding_value=0)  # Pad sequences
    labels = torch.tensor(labels, dtype=torch.float32)
    return texts, labels


In [38]:
print(f"Train data size: {len(train_data)}")
print(f"Test data size: {len(test_data)}")
print(train_data[:2])  # Print first two samples to check structure

Train data size: 25000
Test data size: 25000
[('I mean of all the obscure, overlooked, low budget horror movies waiting to be re-discovered in a DVD release, why pick THE FOREST? I love ultra low budget direct to home video or other alternative release horror. I love 80s hacker horror. I love backwoods slashers with fools wandering off into the night to be chased, murdered and eaten by psychopaths. I am all for the idea of non-professionals working on a horror movie as a way to maybe break into the industry or just making a movie because they want to make one. I am all for using found public locations, non-actors, no name talent behind the camera and in the studio. NO PROBLEM! The most evocative image from THE FOREST is it\'s opening shot of a couple walking in the distance across a forest into the woods: We see them as tiny, vulnerable creatures entering a dank gloomy world where humans may not be the top of the food chain or most feared predator. Then the film takes a dive & never re

In [39]:
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data):
  for text, _ in data:
    yield tokenizer(text)

In [40]:
vocab = build_vocab_from_iterator(yield_tokens(train_data), specials=["<unk>","<pad>"])
vocab.set_default_index(vocab["<unk>"])

In [41]:
def text_pipeline(text):
  return torch.tensor(vocab(tokenizer(text)), dtype = torch.int65)

In [42]:
train_dataset = IMDBDataset(train_data, vocab, tokenizer)
test_dataset = IMDBDataset(test_data, vocab, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_batch)

In [43]:
class SentimentModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(SentimentModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        x = self.fc(hidden[-1])
        return self.sigmoid(x)


In [45]:
from torch.nn.utils.rnn import pad_sequence

In [ ]:
def train(model, train_loader, optimizer, criterion, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0

        for texts, labels in train_loader:
            texts, labels = texts.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(texts).squeeze()  # Raw logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)  # Fix loss accumulation
            predicted = (torch.sigmoid(outputs) > 0.5).float()  # Apply sigmoid before threshold
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        accuracy = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / total:.4f}, Accuracy: {accuracy:.4f}")

def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts).squeeze()  # Raw logits
            predicted = (torch.sigmoid(outputs) > 0.5).float()  # Apply sigmoid
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {correct / total:.4f}")

# Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(vocab)
model = SentimentModel(vocab_size, embed_dim=100, hidden_dim=128).to(device)
criterion = torch.nn.BCEWithLogitsLoss()  # Binary Cross Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Reduced learning rate

# Train and Evaluate
train(model, train_loader, optimizer, criterion, num_epochs=5)
evaluate(model, test_loader)

In [ ]:
import torch

def predict(model, text, tokenizer, vocab, device):
    model.eval()  # Set the model to evaluation mode

    # Tokenize the text and convert tokens to indices
    tokenized_text = tokenizer(text)
    text_tensor = torch.tensor([vocab[token] for token in tokenized_text if token in vocab], dtype=torch.int64).unsqueeze(0).to(device)

    # Forward pass through the model
    with torch.no_grad():
        output = model(text_tensor)

    # Convert output to probability
    prediction = torch.sigmoid(output).item()

    # Convert probability to label
    sentiment = "Positive" if prediction > 0.5 else "Negative"

    return sentiment, prediction

In [ ]:
# Example text
sample_text = "The movie was absolutely fantastic, I loved it!"

# Run prediction
sentiment, confidence = predict(model, sample_text, tokenizer, vocab, device)

# Print the result
print(f"Predicted Sentiment: {sentiment} (Confidence: {confidence:.4f})")